# Decomposição L.U

A decomposição L.U tem o objetivo de decompôr uma matriz $A$ em duas matrizes triangulares, uma superior e uma inferior. Assim, denomina-se L.U (lower upper) este processo.

Entrada: matriz *A*

Saída: matrizes *L* (lower triangular) e *U* (upper triangular)

### Resoluções de sistemas lineares

Para resolver sistemas L.U do tipo $Ax = b$, podemos fazer o seguinte:

$$Ax = b$$
$$(LU)x = b$$
$$Ly = b$$
$$Ux = y$$

In [2]:
import numpy as np

In [15]:
def lu_decomposition(A):
    n = A.shape[0]
    
    U = np.zeros_like(A)
    L = np.zeros_like(A)
    
    for j in range(n):
        L[j][j] = 1.0
        
        for i in range(j+1):
            sum = 0
            for k in range(i):
                sum += U[k][j] * L[i][k]
            U[i][j] = A[i][j] - sum
        
        for i in range(j, n):
            sum = 0
            for k in range(j):
                sum += U[k][j] * L[i][k]
            L[i][j] = (A[i][j] - sum) / U[j][j]
    
    return L, U

In [16]:
def lu_decomposition_partial_pivot(A):
    n = A.shape[0]
    
    U = np.zeros_like(A)
    L = np.zeros_like(A)
    P = np.arange(n)
    
    for j in range(n):
        L[j][j] = 1.0
        
        p = np.argmax(A[j:, j]) + j # A[p, j] is the maximum on array
        if A[p,j] == 0:
            raise Exception('Singular matrix')
            
        P[[j, p]] = P[[p, j]]
        A[[j, p]] = A[[p, j]] # swap rows
        
        for i in range(j+1):
            sum = 0
            for k in range(i):
                sum += U[k][j] * L[i][k]
            U[i][j] = A[i][j] - sum
        
        for i in range(j, n):
            sum = 0
            for k in range(j):
                sum += U[k][j] * L[i][k]
            L[i][j] = (A[i][j] - sum) / U[j][j]
    
    return P, L, U

In [17]:
# A * x = b, em que A é uma matriz triangular inferior
def lower_solve(A, b):
    n = A.shape[0]
    for k in range(1, n):
        b[k] -= A[k, 0:k].dot(b[0:k])
    return b

In [18]:
# A * x = b, em que A é uma matriz triangular superior
def upper_solve(A, b):
    n = A.shape[0]
    for k in range(n-1, -1, -1):
        b[k] -= A[k, k+1:n].dot(b[k+1:n])
        b[k] /= A[k,k]
    return b

In [19]:
def lu_solve(A, b):
    L, U = lu_decomposition(A)
    y = lower_solve(L, b)
    x = upper_solve(U, y)
    return x

In [20]:
def lu_solve_partial_pivot(A, b):
    _, L, U = lu_decomposition_partial_pivot(A)
    y = lower_solve(L, b)
    x = upper_solve(U, y)
    return x

In [21]:
A = np.array([[1, 2, 1], [2, 5, 3],[1, 4, 9]]).astype('float64')
b = np.array([3,5,-1]).astype('float64')

P, L, U = lu_decomposition_partial_pivot(A)
print(P)
print(L)
print(U)

[1 2 0]
[[ 1.          0.          0.        ]
 [ 0.5         1.          0.        ]
 [ 0.5        -0.33333333  1.        ]]
[[2.  5.  3. ]
 [0.  1.5 7.5]
 [0.  0.  2. ]]


In [22]:
A = np.array([[1, 2,0], [1, 3,1],[-2, 0,1]]).astype('float64')
b = np.array([3,5,-1]).astype('float64')

L, U = lu_decomposition(A)
print(L)
print(U)
print(lu_solve(A, b))

[[ 1.  0.  0.]
 [ 1.  1.  0.]
 [-2.  4.  1.]]
[[ 1.  2.  0.]
 [ 0.  1.  1.]
 [ 0.  0. -3.]]
[1. 1. 1.]
